Previously, we have seen how to analyze and manipulate data in a single DataFrame. However, you will often find data saved into different files and, therefore, you may need to deal with several different pandas DataFrames.<br>
In this session we will explore different ways of combining DataFrames into a single DataFrame.

Let's start loading the `pandas` library, reading two data sets into DataFrames, and having a quick look at the tabular data: ```surveys.csv``` and ```species.csv```

In [ ]:
import pandas as pd

In [ ]:
surveys_df = pd.read_csv("../course_materials/data/surveys.csv")
species_df = pd.read_csv("../course_materials/data/species.csv")

In [ ]:
print(surveys_df.info())
surveys_df.head()

In [ ]:
print(species_df.info())
species_df.head()

We now have two DataFrames. The first, `surveys_df`, contains information on individuals of a species recorded in a survey, while the second, `species_df`, contains more detailed information on each species.

## Concatenating DataFrames

The first way we will combine DataFrames is **concatenation**, i.e. simply putting DataFrames one after the other either **vertically** or **horizontally**.

Concatenation can be used if the DataFrames are similar, meaning that they either have the same rows or columns. We will see examples of this later.

To concatenate two DataFrames you will use the function ```pd.concat()```, specifying as arguments the DataFrames to concatenate and ```axis=0``` or ```axis=1``` for vertical or horizontal concatenation, respectively.

We will only be looking at vertical concatenation, but it is good to be aware that there is more to be discovered beyond what we describe here.

Let us first obtain two small DataFrames from the larger `surveys_df` dataset.

In [ ]:
# Subsetting DataFrames
surveys_df_sub_first10 = surveys_df.head(10)
surveys_df_sub_last10  = surveys_df.tail(10)

We now have two DataFrames, one with the first ten rows of the original dataset, and another with the last ten rows.

Now, let us do some **vertical concatenation** or **stacking**. In this case the two DataFrames are simply stacked 'on top of' each other (remember to specify ```axis=0```).
<div>
<img src="images/vertical_stacking.jpeg" width="300"/>
</div>
Vertical stacking can be understood as combining two DataFrames that have different sets of the same type of data. In our example, it may be that one field researcher has registered the first ten entries, and another did the last ten, both using the same laboratory sheets. They both wrote down the same information (weight, species, and so on) of all different individuals. If we combine them, we have one list of twenty records, rather than two lists of ten. 

In [ ]:
# Stack the DataFrames on top of each other
vertical_stack = pd.concat([surveys_df_sub_first10, surveys_df_sub_last10], axis=0)

In [ ]:
print(vertical_stack.info())
vertical_stack

The resulting DataFrame (```vertical_stack```) consists, as expected, of 20 rows. These are the result of the first and last 10 rows of our original DataFrame ```surveys_df```.

You may have noticed that the last ten rows have very high index, not consecutive with the first ten rows. This is because concatenation preserves the indices of the two original DataFrames. If you want a brand new set of indices for your concateneted DataFrame, simply reset the indices using the method ```.reset_index()```. Notice that this adds a column `index` to your DataFrame, that maintains the original index. If you pass `drop=True` into the function, you will avoid the addition of this column.

In [ ]:
vertical_stack.reset_index()

## Joining DataFrames

Concatenating DataFrames allows you to combine two entire DataFrames into a single one. In many cases, you want to combine only selected parts of two DataFrames.

You might, for example, want to merge rows of two DataFrames that have matching values in specific columns. The pandas function ```merge()``` performs an operation that you may know as a *join* if you worked with databases before. The join operation joins the content of two DataFrames in a particular way. There are different types of joins, but the workflow to perform a join operation is always the same:

  1. You identify a *left* and a *right* DataFrame, among the two you want to join;
  2. You identify in both your left and right DataFrame a column (or set of columns) to join on;
  3. You choose the type of join;
  4. You perform the join running the function `pd.merge()` with the specified inputs and options.

What it means for a DataFrame to be 'left' or 'right' depends on the type of join. The main thing to remember is that it matters which DataFrame you mention first when performing a join.

We will only be looking at what is know as an *inner join*. Investigating the other types of joins is left as an exercise.

Let's see some join example using the first ten rows of our `surveys_df` DataFrame (`surveys_df_sub_first10`). This will be our left DataFrame, containing data on individual animals. Our right DataFrame will be the first 20 rows of `species_df`, containing some information on each species. Let us rename them for the purpose of the join:

In [ ]:
left_df = surveys_df_sub_first10
right_df = species_df.head(20)

In [ ]:
left_df

In [ ]:
right_df

The column we want to perform the join on is the one containing information about the `species_id`. Conveniently, this column has the same label in both DataFrames. Note that this is not always the case.

In [ ]:
inner_join = pd.merge(left_df, right_df, left_on='species_id', right_on='species_id', how='inner')
inner_join

As you may notice, the resulting DataFrame has only seven rows, while our original DataFrames had 10 and 20 rows, respectively. This is because an inner join selects only those rows where the value of the joined column occurs in both DataFrames (mathematically, an intersection).

Aside from the inner join, there are three more types of join that you can do using the `merge()` function:
- An inner join selects only the rows that result from the combination of matching rows in both the original left and right DataFrames (intersection);
- A left join selects all rows that were in the original left DataFrame, some of which may have been joined with a matching entry from the right DataFrame;
- A right join selects all rows that were in the original right DataFrame, some of which may have been joined with a matching entry from the left DataFrame;
- An outer join merges the two DataFrames and keeps all resulting rows.

To better understand how join works, it may be useful to look at the diagrams below:

<div>
<img src="images/joins.jpeg" width="600"/>
</div>

  - Do you want to select only **common** information between the two DataFrames? Then you use an **inner join**;
  - Do you want to **add information** to your **left** DataFrame? Then you use a **left join**;
  - Do you want to **add information** to your **right** DataFrame? Then you use a **right join**;
  - Do you want to get **all the information** from the two DataFrames? Then you use an **outer join**.

<div class="alert alert-block alert-success">
<b>Exercise 12</b>
    
Now go to the Jupyter Dashboard in your internet browser and continue with exercise 10 and 11.


We will continue with [Data visualization](data-science-with-pandas-4.ipynb).